# Importing Libraries

In [33]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import datetime
import warnings
warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('dark_background')
from tqdm import tqdm_notebook

In [34]:
tf.test.is_built_with_cuda()

True

# Defining Classes & Generators


In [35]:
classes = ['Knife','Pistol']

In [36]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.0, 
                               shear_range=0.15, 
                               zoom_range=0.2, 
                               horizontal_flip=True) 

train_data = generator.flow_from_directory( directory='/kaggle/input/knife-vs-pistol/Knife_vs_Pistol', 
                                            target_size=(128, 128),
                                            classes=classes, 
                                            batch_size=128)

test_data = generator.flow_from_directory( directory='/kaggle/input/knife-vs-pistol/Knife_vs_Pistol', 
                                           target_size=(128, 128), 
                                           classes=["eval_Knife","eval_pistol"], 
                                           batch_size=128,
                                           shuffle=False)

Found 414 images belonging to 2 classes.
Found 26 images belonging to 2 classes.


# MobileNetV2

### Loading the Model

In [37]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3),include_top=False,weights="imagenet")
# base_model.summary()

In [38]:
base_model.trainable = False
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

### 2 Apporaches can be taken
   1. Using Flattening Layer
   2. Using GlobalAveragePooling Layer

In [39]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d_1')>

In [40]:
pred_layer = tf.keras.layers.Dense(units=2,activation="sigmoid")(global_average_layer)
pred_layer

<KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'dense_1')>

In [41]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=pred_layer)
# model.summary()

In [42]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.01),loss="categorical_crossentropy",metrics=["accuracy"])

In [43]:
earlyStopping =  tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_accuracy',restore_best_weights=True,start_from_epoch=2)

# history = model.fit_generator(train_data, epochs=5, validation_data=test_data,callbacks=[earlyStopping])
history = model.fit(train_data, epochs=10, validation_data=test_data, shuffle=True, callbacks=[earlyStopping],batch_size=64)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7b0bc3693380>

In [ ]:
df = pd.DataFrame(history.history)
df_loss = df[['loss', 'val_loss']]
df_acc =  df[['accuracy', 'val_accuracy']]

In [ ]:
plt.figure(figsize=(10, 4))
plt.title("Loss & Val Loss")
sns.lineplot(df_loss,palette="flare");

plt.figure(figsize=(10, 4))
plt.title("Accuracy & Val Accuracy")
sns.lineplot(df_acc,palette="flare");